202010370311391 - Ungga Putra Mahendra

In [ ]:
pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
pip install transformers torch pandas

In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import numpy as np
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_excel('Overview.xlsx')

In [ ]:
df

,No,No Putusan,Barang Bukti,Amar Putusan
0,1,94/Pid.Sus/2022/PN Mad,1 (satu) kantong plastic klip berisi narkotika...,Menyatakan Terdakwa ARIS KURNIAWAN Als IPIN Al...
1,2,70/Pid.Sus/2022/PN Mad,1 (satu) unit ponsel merk Xiaomi warna hitam k...,Menyatakan Terdakwa Lutfi Maulana Rosida Alias...
2,3,9/Pid.Sus/2022/PN Mad,1 ( satu ) bungkusan alumunium foil warna silv...,Menyatakan Terdakwa JIMMY WAHYU PRANOTO Als. J...
3,4,139/Pid.Sus/2022/PN Mad,1 (satu) buah plastik klip didalamnya berisi s...,enyatakan Terdakwa I Jevian Andika Wardana als...
4,5,141/Pid.Sus/2022/PN Mad,1 (satu) unit Ponsel merk Vivo warna hitam kom...,Menyatakan Terdakwa I HARI SUPRIYANTO BIN KADI...
...,...,...,...,...
62,63,61/Pid.Sus/2022/PN Mad,1 (satu) potong sedotan plastik warna hijau di...,Menyatakan Terdakwa ARMY SANCOYO KUNTORO Bin S...
63,64,17/Pid.Sus/2022/PN Mad,1 (satu) buah tas warna biru merk Forester ber...,Menyatakan Terdakwa NOVI LUKIANTO AliasKENTRUN...
64,65,\t12/Pid.Sus/2022/PN Mad,1 (satu) buah replika buaya warna biru didalam...,Menyatakan Terdakwa CANDRA MEI YANTO BATI Bin ...
65,66,50/Pid.Sus/2022/PN Mad,1 (satu) gulungan lakban yang didalamnya terda...,Menyatakan Terdakwa ARJUN DICKY PRASETYO AGUNG...


In [ ]:
df_copy = df.copy()

In [ ]:
def preprocess_text(text):
    # Cek jika value adalah string
    if isinstance(text, str):
        # Menghilangkan angka
        text = ''.join([i for i in text if not i.isdigit()])

        # Proses case folding (lowercasing)
        text = text.lower()

        # Proses stemming
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        text = stemmer.stem(text)

        # Proses stop word removal
        factory = StopWordRemoverFactory()
        stopword = factory.create_stop_word_remover()
        text = stopword.remove(text)

    return text

In [ ]:
df_copy['BarangBukti_preprocessed'] = df_copy['Barang Bukti'].apply(preprocess_text)
df_copy['AmarPutusan_preprocessed'] = df_copy['Amar Putusan'].apply(preprocess_text)

In [ ]:
df_copy

,No,No Putusan,Barang Bukti,Amar Putusan,BarangBukti_preprocessed,AmarPutusan_preprocessed
0,1,94/Pid.Sus/2022/PN Mad,1 (satu) kantong plastic klip berisi narkotika...,Menyatakan Terdakwa ARIS KURNIAWAN Als IPIN Al...,satu kantong plastic klip isi narkotika jenis ...,nyata dakwa aris kurniawan als ipin als memble...
1,2,70/Pid.Sus/2022/PN Mad,1 (satu) unit ponsel merk Xiaomi warna hitam k...,Menyatakan Terdakwa Lutfi Maulana Rosida Alias...,satu unit ponsel merk xiaomi warna hitam kombi...,nyata dakwa lutfi maulana rosida alias kepek a...
2,3,9/Pid.Sus/2022/PN Mad,1 ( satu ) bungkusan alumunium foil warna silv...,Menyatakan Terdakwa JIMMY WAHYU PRANOTO Als. J...,satu bungkus alumunium foil warna silver isi s...,nyata dakwa jimmy wahyu pranoto als jimbek bin...
3,4,139/Pid.Sus/2022/PN Mad,1 (satu) buah plastik klip didalamnya berisi s...,enyatakan Terdakwa I Jevian Andika Wardana als...,satu buah plastik klip isi serbuk kristal warn...,enyatakan dakwa i jevian andika wardana als je...
4,5,141/Pid.Sus/2022/PN Mad,1 (satu) unit Ponsel merk Vivo warna hitam kom...,Menyatakan Terdakwa I HARI SUPRIYANTO BIN KADI...,satu unit ponsel merk vivo warna hitam kombina...,nyata dakwa i hari supriyanto bin kadirun alm ...
...,...,...,...,...,...,...
62,63,61/Pid.Sus/2022/PN Mad,1 (satu) potong sedotan plastik warna hijau di...,Menyatakan Terdakwa ARMY SANCOYO KUNTORO Bin S...,satu potong sedot plastik warna hijau dalam na...,nyata dakwa army sancoyo kuntoro bin soekidi b...
63,64,17/Pid.Sus/2022/PN Mad,1 (satu) buah tas warna biru merk Forester ber...,Menyatakan Terdakwa NOVI LUKIANTO AliasKENTRUN...,satu buah tas warna biru merk forester isi sat...,nyata dakwa novi lukianto aliaskentrung bin su...
64,65,\t12/Pid.Sus/2022/PN Mad,1 (satu) buah replika buaya warna biru didalam...,Menyatakan Terdakwa CANDRA MEI YANTO BATI Bin ...,satu buah replika buaya warna biru isi satu ka...,nyata dakwa candra mei yanto bati bin bambang ...
65,66,50/Pid.Sus/2022/PN Mad,1 (satu) gulungan lakban yang didalamnya terda...,Menyatakan Terdakwa ARJUN DICKY PRASETYO AGUNG...,satu gulung lakban dalam satu kantong plastik ...,nyata dakwa arjun dicky prasetyo agung bin sug...


In [ ]:
columns_to_drop = ['Barang Bukti', 'Amar Putusan']
df_copy = df_copy.drop(columns=columns_to_drop)
df_copy

,No,No Putusan,BarangBukti_preprocessed,AmarPutusan_preprocessed
0,1,94/Pid.Sus/2022/PN Mad,satu kantong plastic klip isi narkotika jenis ...,nyata dakwa aris kurniawan als ipin als memble...
1,2,70/Pid.Sus/2022/PN Mad,satu unit ponsel merk xiaomi warna hitam kombi...,nyata dakwa lutfi maulana rosida alias kepek a...
2,3,9/Pid.Sus/2022/PN Mad,satu bungkus alumunium foil warna silver isi s...,nyata dakwa jimmy wahyu pranoto als jimbek bin...
3,4,139/Pid.Sus/2022/PN Mad,satu buah plastik klip isi serbuk kristal warn...,enyatakan dakwa i jevian andika wardana als je...
4,5,141/Pid.Sus/2022/PN Mad,satu unit ponsel merk vivo warna hitam kombina...,nyata dakwa i hari supriyanto bin kadirun alm ...
...,...,...,...,...
62,63,61/Pid.Sus/2022/PN Mad,satu potong sedot plastik warna hijau dalam na...,nyata dakwa army sancoyo kuntoro bin soekidi b...
63,64,17/Pid.Sus/2022/PN Mad,satu buah tas warna biru merk forester isi sat...,nyata dakwa novi lukianto aliaskentrung bin su...
64,65,\t12/Pid.Sus/2022/PN Mad,satu buah replika buaya warna biru isi satu ka...,nyata dakwa candra mei yanto bati bin bambang ...
65,66,50/Pid.Sus/2022/PN Mad,satu gulung lakban dalam satu kantong plastik ...,nyata dakwa arjun dicky prasetyo agung bin sug...


In [ ]:
df_copy.isna().sum()

No                          0
No Putusan                  0
BarangBukti_preprocessed    1
AmarPutusan_preprocessed    0
dtype: int64

In [ ]:
df_new = df_copy.dropna()

In [ ]:
df_new

,No,No Putusan,BarangBukti_preprocessed,AmarPutusan_preprocessed
0,1,94/Pid.Sus/2022/PN Mad,satu kantong plastic klip isi narkotika jenis ...,nyata dakwa aris kurniawan als ipin als memble...
1,2,70/Pid.Sus/2022/PN Mad,satu unit ponsel merk xiaomi warna hitam kombi...,nyata dakwa lutfi maulana rosida alias kepek a...
2,3,9/Pid.Sus/2022/PN Mad,satu bungkus alumunium foil warna silver isi s...,nyata dakwa jimmy wahyu pranoto als jimbek bin...
3,4,139/Pid.Sus/2022/PN Mad,satu buah plastik klip isi serbuk kristal warn...,enyatakan dakwa i jevian andika wardana als je...
4,5,141/Pid.Sus/2022/PN Mad,satu unit ponsel merk vivo warna hitam kombina...,nyata dakwa i hari supriyanto bin kadirun alm ...
...,...,...,...,...
62,63,61/Pid.Sus/2022/PN Mad,satu potong sedot plastik warna hijau dalam na...,nyata dakwa army sancoyo kuntoro bin soekidi b...
63,64,17/Pid.Sus/2022/PN Mad,satu buah tas warna biru merk forester isi sat...,nyata dakwa novi lukianto aliaskentrung bin su...
64,65,\t12/Pid.Sus/2022/PN Mad,satu buah replika buaya warna biru isi satu ka...,nyata dakwa candra mei yanto bati bin bambang ...
65,66,50/Pid.Sus/2022/PN Mad,satu gulung lakban dalam satu kantong plastik ...,nyata dakwa arjun dicky prasetyo agung bin sug...


#Model - Naive Bayes

In [ ]:
# Fungsi untuk preprocessing teks
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

In [ ]:
# Preprocessing teks
df_new['BarangBukti_preprocessed'] = df_new['BarangBukti_preprocessed'].apply(preprocess_text)
df_new['AmarPutusan_preprocessed'] = df_new['AmarPutusan_preprocessed'].apply(preprocess_text)

<ipython-input-167-d5218a595cf4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['BarangBukti_preprocessed'] = df_new['BarangBukti_preprocessed'].apply(preprocess_text)
<ipython-input-167-d5218a595cf4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['AmarPutusan_preprocessed'] = df_new['AmarPutusan_preprocessed'].apply(preprocess_text)


In [ ]:
df = df_new.dropna()

In [ ]:
# Membuat model Naive Bayes untuk mengukur kemiripan
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(df['BarangBukti_preprocessed'] + ' ' + df['AmarPutusan_preprocessed'], df['No'])

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [ ]:
# User input untuk query
user_query = input("Masukkan query: ")
user_query = preprocess_text(user_query)

Masukkan query: xiaomi


In [ ]:
# Transform the query using the TfidfVectorizer
query_vector = model.named_steps['tfidfvectorizer'].transform([user_query])

In [ ]:
# Transform the documents in the dataset
document_vectors = model.named_steps['tfidfvectorizer'].transform(df['BarangBukti_preprocessed'] + ' ' + df['AmarPutusan_preprocessed'])

In [ ]:
# Measure cosine similarity between the query and documents
similarities = cosine_similarity(query_vector, document_vectors)

In [ ]:
# Find the indices of the top 5 documents with the highest similarity
top_indices = similarities.argsort()[0, ::-1][:5]

In [ ]:
# Display the top 5 documents
print("\nTop 5 Documents:")
for i, index in enumerate(top_indices, 1):
    top_document = df.iloc[index]
    print(f"\nDocument {i}:")
    print("No Putusan:", top_document['No Putusan'])
    print("BarangBukti_preprocessed:", top_document['BarangBukti_preprocessed'])
    print("AmarPutusan_preprocessed:", top_document['AmarPutusan_preprocessed'])

# Model evaluation metrics
ground_truth_labels = df['No']
predicted_labels = model.predict(df['BarangBukti_preprocessed'] + ' ' + df['AmarPutusan_preprocessed'])

# Calculate and display evaluation metrics
accuracy = accuracy_score(ground_truth_labels, predicted_labels)
f1 = f1_score(ground_truth_labels, predicted_labels, average='weighted')  # Use 'weighted' for multiclass problems
recall = recall_score(ground_truth_labels, predicted_labels, average='weighted')
conf_matrix = confusion_matrix(ground_truth_labels, predicted_labels)

# Display the model evaluation metrics
print("\nModel Evaluation Metrics:")
print("Accuracy:", accuracy)
print("F1-score:", f1)
print("Recall:", recall)
print("Confusion Matrix:")
print(conf_matrix)


Top 5 Documents:

Document 1:
No Putusan: 70/Pid.Sus/2022/PN Mad
BarangBukti_preprocessed: unit ponsel merk xiaomi warna hitam kombinasi biru nomor whatsapp
AmarPutusan_preprocessed: nyata dakwa lutfi maulana rosida alias kepek alias petal bin purnomo bukti sah salah laku tindak pidana laku tindak pidana hak jual beli narkotika golong i maksud ayat bentuk tanam berat kilogram batang pohon bentuk tanam berat gram dakwa jatuh pidana dakwa pidana penjara delapan denda rp delapan ratus juta rupiah denda bayar ganti pidana penjara puluh barang bukti upa unit ponsel merk xiaomi warna hitam kombinasi biru nomor whatsapp musnah beban dakwa bayar biaya perkara rp ribu rupia

Document 2:
No Putusan: 136/Pid.Sus/2022/PN Mad
BarangBukti_preprocessed: buah potong pipa paralon bungkus lakban coklat kantong klip palstik isi narkotika jenis shabu berat kotor gram netto gram kantong klip palstik isi narkotika jenis shabu berat kotor koma tujuh gram netto gram kantong klip palstik isi narkotika jenis s